In [1]:
import tensorflow as tf
from tqdm import trange, tqdm_notebook
import os
tf.logging.set_verbosity(tf.logging.ERROR)
# 기본값은 0 (모든 로그가 표시됨) INFO 로그를 필터링하려면 1, 
# WARNING 로그를 필터링하려면 2, ERROR 로그를 추가로 필터링하려면 3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# image = 28*28*1 = 784*1
X = tf.placeholder(tf.float32, [None, 784]) # [# of batch data, # of features(columns)]
Y = tf.placeholder(tf.float32, [None, 10]) # 0~9 

# Sess.run 할때 옵션값으로 줘야한다. 
keep_prob = tf.placeholder(tf.float32) # 살려줄 node의 비율
bn_sign = tf.placeholder(tf.bool)

########### 1레이어 #############
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
# BN단계, training단계 test단계일때 true,false 전환 해줘야함
L1 = tf.layers.batch_normalization(L1, training=bn_sign)
# 드랍아웃 비율 keep_prob = 0.8 , 80%를 살리겠다
L1 = tf.nn.dropout(L1, keep_prob)

########### 2레이어 #############
W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))
L2 = tf.layers.batch_normalization(L2, training=bn_sign)
L2 = tf.nn.dropout(L2, keep_prob)

########### 마지막 레이어 출력값 10개. ############## 
W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

## 손실함수
cost = tf.losses.softmax_cross_entropy(Y, model) # for Classification

#optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# tf 1.xx 버전에서 BN을 사용할땐 옵티마이저를 이렇게 선언해야한다. 보통경우는 위의 주석
## 최적화
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(cost)

# global_variables_initializer 하는 이유.
# 텐서플로의 연산은 데이터 플로우 그래프로 구성된다.
# 텐서플로에서 변수형, 앞서 선언한 레이어들은 
# 그래프를 실행하기 전에(sess.run) 초기화를 해줘야 값이 지정이 된다. 
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
print(total_batch)

########## 훈련 ###########
# tqdm을 이용하면 바 형태로 진행상태를 볼 수 있다.
for epoch in tqdm_notebook(range(15)):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        # gradient descent
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, 
                                                             Y: batch_ys,
                                                             keep_prob: 0.8,
                                                             bn_sign: True})
        total_cost += cost_val
    # 테스트는 훈련할 필요가 없으니 optimizer는 불필요
    test_cost = sess.run([cost], feed_dict={X: mnist.test.images, Y: mnist.test.labels, 
                                            keep_prob: 1.0, bn_sign: False}) # current test error
    
    print('Epoch: {}'.format(epoch+1), 
          '|| Avg. Training cost = {:.3f}'.format(total_cost / total_batch),
          '|| Current Test cost = {:.3f}'.format(test_cost[0]))

print('Learning process is completed!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1)) # model : 예측값, Y : 실제 정답
# True or False의 비율을 MSE로 구한다. ex) 98true, 2false = 0.98 accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도 :', sess.run(accuracy,
                        feed_dict={X: mnist.test.images,
                                   Y: mnist.test.labels,
                                   keep_prob: 1.0,
                                   bn_sign: False}))

# 모델 예측 결과값
# 모델의 반환값이 [ 1 , -2 , 3 , 4 , -5 , 0 , 20, -2 , -15, 5 ] 이렇게 나오는데 
# 이 경우 6번째가 가장 크니 6번째를 반환 해야한다. 
# tf.argmax(model, 1) 은 그 반환 값을 정해주는것
predicted_labels = sess.run(tf.argmax(model, 1), feed_dict={X: mnist.test.images, Y: mnist.test.labels, 
                                                            keep_prob: 1.0, bn_sign: False})
# 예측한 값들을 담은 리스트 앞에서 10개 출력.
print(list(predicted_labels)[:10])
# 실제 정답 
import numpy as np
print(np.argmax(mnist.test.labels, 1)[:10])

550


Epoch: 1 || Avg. Training cost = 0.228 || Current Test cost = 0.116
Epoch: 2 || Avg. Training cost = 0.113 || Current Test cost = 0.090
Epoch: 3 || Avg. Training cost = 0.090 || Current Test cost = 0.084
Epoch: 4 || Avg. Training cost = 0.074 || Current Test cost = 0.072
Epoch: 5 || Avg. Training cost = 0.067 || Current Test cost = 0.070
Epoch: 6 || Avg. Training cost = 0.057 || Current Test cost = 0.074
Epoch: 7 || Avg. Training cost = 0.054 || Current Test cost = 0.072
Epoch: 8 || Avg. Training cost = 0.049 || Current Test cost = 0.064
Epoch: 9 || Avg. Training cost = 0.042 || Current Test cost = 0.062
Epoch: 10 || Avg. Training cost = 0.041 || Current Test cost = 0.065
Epoch: 11 || Avg. Training cost = 0.037 || Current Test cost = 0.066
Epoch: 12 || Avg. Training cost = 0.039 || Current Test cost = 0.068
Epoch: 13 || Avg. Training cost = 0.033 || Current Test cost = 0.059
Epoch: 14 || Avg. Training cost = 0.033 || Current Test cost = 0.064
Epoch: 15 || Avg. Training cost = 0.032 || 

Fully-Connected NN
Dense Network
Feed forward NN